In [41]:
from openai import AzureOpenAI
import yaml

with open('openai_gpt.yaml') as f:
    config = yaml.safe_load(f)

api_version = config["config"]["api_version"]
azure_endpoint = config["config"]["azure_endpoint"]
api_key = config["config"]["api_key"]

user_input_1 = "꽃냄새가 좋아 특히 장미 같은 분홍색 꽃향기, 자연스러운 꽃냄새와 그리고 포근한 향이 좋아. 살냄새가 부드럽게 나는 것! 달콤한 베리류도 좋아."
user_input_2 = "샤넬 레젝 1957 향수를 좋아해. 그리고 클린 향수들은 다 내 취향인듯"

In [42]:
client = AzureOpenAI(
    api_version = api_version,
    azure_endpoint = azure_endpoint,
    api_key = api_key
)   

In [43]:
response = client.chat.completions.create(
    model = "hatcheryOpenaiCanadaGPT4o",
    messages=[
        {"role": "system", "content": """

넌 '사용자'가 좋아하는 향기를 그룹으로 분류하는 역할을 해.
'사용자'의 입력을 이해하고, 좋아한다고 이야기한 향기/냄새를 아래 12개의 그룹 중 하나로 각각 매핑해줘.
답변은 그룹의 리스트만 출력해줘.
각 그룹이 어떤 향기/냄새들을 의미하는지 읽고 사용자가 좋아하는 향을 그룹으로 매핑해줘.
        
<group>
-그룹 : ['animal', 'balsamic', 'beverage', 'citrus', floral', 'fruit', 'green', 'spicy', 'sweet', 'synthetic', 'white_floral', 'woody']
1. animal : 가죽, 모피 등 동물에서 유래한 향조로, 포근한 살냄새, 풍부하고 따뜻하며 감각적인 느낌, 예시) 엠버, 머스크계열, 살냄새, 암브레트, 등
2. balsamic : 부드럽고 따뜻한 향조로, 달콤하고 크리미한 느낌을 주며 오리엔탈 향, 예시) 벤조인, 수지, 레진류
3. beverage : 커피, 와인, 차와 같은 음료에서 영감을 받은 향조로, 독특하면서도 깊이 있는 향, 예시) 술종류, 차종류, 음료종류
4. citrus : 주로 레몬, 오렌지, 자몽 등 감귤류에서 추출된 상큼하고 밝은 향, 신선하고 시원하고 상쾌한 향, 예시) 베르가못, 오렌지류, 자몽류, 레몬류, 귤류, 유자, 라임 등
5. floral : 꽃에서 추출된 향조로, 우아하고 여성스러운 향, 예시) 꽃종류, 장미, 라일락 등 
6. fruit : 과일에서 나온 향조로, 달콤하고 상큼한 과일의 향, 예시) 베리류, 복숭아, 사과 등
7. green : 풀, 나뭇잎, 허브 등 자연의 푸른 향을 담은 향조로, 신선하고 상쾌하며 자연스러운 느낌, 예시) 풀종류, 잎종류 등
8. spicy : 계피, 정향, 후추와 같은 향신료에서 나온 향조로, 따뜻하고 톡 쏘는 느낌, 강렬함, 예시) 계피, 정향, 후추, 바닐라 등 
9. sweet : 설탕, 꿀, 캐러멜 같은 달콤한 향조로, 부드럽고 풍부한 향, 예시) 디저트류, 설탕, 꿀, 캐러맬 등 단 음식류
10. synthetic : 파우더리, 알데하이드, 담배, 메탈릭, aeric, 종이, 물 등 순수 자연물 외의 향기 
11. white_floral : 가드니아, 튜베로즈 같은 흰 꽃에서 추출된 향조로, 깨끗하고 순수하며 풍성한 꽃향기, 예시) 오렌지꽃, 자스민, 백함 등
12. woody : 샌달우드, 시더우드 같은 나무에서 추출된 향조로, 따뜻하고 안정감을 주며 깊이 있는 향, 예시) 나무종류, 절간냄새 등
</group>

<example>
'사용자'의 입력 예시: "난 상큼한 향을 좋아해. 레몬이나 자몽같은.. 그리고 포근한 냄새를 좋아하는 것 같아. 주로 머스크 계열! 그리고 화이트 플로럴 계열이 좋아"
답변 예시: ['citrus', 'animal', 'white_floral']
</example> """},
        {"role": "user", "content": f"""
<input>
-'사용자'의 입력 : {user_input_1}
</input>
         """}   
    ]
)

preference = response.choices[0].message.content
print(preference)

['floral', 'animal', 'sweet']


In [44]:
response = client.chat.completions.create(
    model = "hatcheryOpenaiCanadaGPT4o",
    messages=[
        {"role": "system", "content": """

우리는 향수 brand와 title 정보를 담은 Database가 있어. 
'사용자'가 오래 사용했던 향수나 좋아하는 향수 브랜드를 입력으로 제시할 거야.
'사용자'의 입력을 Database에 검색할 수 있도록 Fragrantica에서 사용하는 영어 brand와 title으로 정제하는 역할을 해줘.
잘 모르겠으면 웹 검색을 통해 정확한 정보를 제공해줘.
'사용자'는 brand와 title 중 하나만 입력할 수도 있으니 '사용자'가 제시하지 않는 정보는 None 처리해줘.
답변은 향수 정보를 담은 리스트만 출력해줘.

<example1>
'사용자'의 입력 예시: "나는 메종프란시스커정의 바카라루쥬를 오래썼어. 또 딥디크 오데썽도 좋아해."
답변 예시: [['Maison-Francis-Kurkdjian', 'Baccarat Rouge 540 Maison Francis Kurkdjian for women and men'], ['Diptyque', 'Eau des Sens Diptyque for women and men']]
</example1>

<example2>
'사용자'의 입력 예시: "조말론의 블루베리 향수를 좋아해. 그리고 대체로 캘빈클라인 향수들은 나랑 잘 맞는 것 같아"
답변 예시: [['Jo-Malone-London', 'Blackberry & Bay Jo Malone London for women'], ['Calvin-Klein', None]]
</example2>

<example3>
'사용자'의 입력 예시: "네롤리포르토피노라는 향수를 좋아하는데 브랜드는 기억 안나"
답변 예시: [['Tom-Ford', 'Neroli Portofino Acqua Tom Ford for women and men']]
</example3>  """},
        
        {"role": "user", "content": f"""
<input>
-'사용자'의 입력 : {user_input_2}
</input>
         """}   
    ]
)

history = response.choices[0].message.content
print(history)

[['Chanel', 'Les Exclusifs de Chanel 1957 Chanel for women and men'], ['Clean', None]]


### RAG

In [21]:
import pandas as pd
import numpy as np
import os
import openai
from tqdm import tqdm   

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [23]:
# DB 클라이언트 생성 
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(host='localhost', port=8000)

# 컬렉션 로드 
perfume_name = client.get_collection("perfume_name")
perfume_note = client.get_collection("perfume_note")
perfume_dist = client.get_collection("perfume_dist")

In [24]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# 임베딩 함수 정의
def embedding_function(texts):
    return model.encode(texts)

In [35]:
history

"[['Chanel', 'Les Exclusifs de Chanel 1957 Chanel for women and men'], ['Clean', None]]"

In [45]:
##### 과거 이력 기반 

import ast

historys = ast.literal_eval(history)
sim_ids = []

for hist in historys:
    
    if hist[1]:
        emb = embedding_function(hist[1])
        name = perfume_name.query(
        embeddings,
        n_results=1
        )
        sim_ids.extend(name['ids'][0])
    
    else:
        pass
        
sim_ids
# documents = ['Jo malone']
# embeddings = model.encode(documents)

# perfume_name.query(
#     embeddings,
#     n_results=3
# )

['13793']

In [27]:
documents = ['Les Exclusifs de Chanel 1957 Chanel for women and men']
embeddings = model.encode(documents)

perfume_name.query(
    embeddings,
    n_results=3
)

perfume_name.get(
    ids = ['1']
)

{'ids': ['1'],
 'embeddings': None,
 'metadatas': [None],
 'documents': ['Aatifa Ajmal for women and men'],
 'data': None,
 'uris': None,
 'included': ['metadatas', 'documents']}

In [ ]:
##### 취향 기반 

documents = ['Les Exclusifs de Chanel 1957 Chanel for women and men']
embeddings = model.encode(documents)

perfume_name.query(
    embeddings,
    n_results=3
)